In [244]:
import pandas as pd
import numpy as np
import json
import os
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from sklearn.metrics import confusion_matrix

from IPython.display import display
pd.set_option('display.max_rows',5)
pd.set_option('precision',3)

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important;}</style>")

In [70]:
def json_to_df(file_name):
    #print(file_name)
    with open(file_name, encoding='utf-8', errors='ignore') as f:
        json_1 = json.load(f)
    dict_1 = dict(json_1)
    df_1 = pd.DataFrame.from_dict(dict_1)
    return df_1
def create_padded_docs(t,doc,max_length = 500):
    encoded_docs = t.texts_to_sequences(doc)
    #print(encoded_docs)
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    #print(padded_docs)
    return padded_docs

In [403]:
all_data_df['Id']=all_data_df.index
all_data_df

,index,text,sentiment,Id
0,1,Could you catch us up on your latest thinking ...,neutral,0
1,10,We continue to ramp up the activities for the ...,neutral,1
...,...,...,...,...
1642,98,And so it's -- that one is a little bit toughe...,positive,1642
1643,99,"Thanks, Simeon. Rob, next question please?",neutral,1643


In [406]:
import pickle
filename = 'all_group_data.pickle'
outfile = open(filename,'wb')
pickle.dump(all_data_df,outfile)
outfile.close()

In [373]:
all_data=[]
for file in os.listdir():
    if file.startswith("Team") and file.endswith(".json"):
        df=json_to_df(file)
        df.columns=['text','sentiment']
        df['sentiment']=df['sentiment'].apply(lambda x: x.lower())
        if sum(df.isnull().any(axis=1))>0:
            print(file)
            display(df[pd.isnull(df['Sentiment'])])
        else:
            all_data.append(df)
all_data_df=pd.concat(all_data)
all_data_df['sentiment']=all_data_df['sentiment'].apply(lambda x: 'neutral' if x in ['neutra;','negetive','neural'] else x)
all_data_df['sentiment']=all_data_df['sentiment'].apply(lambda x: 'positive' if x in ['postive','positivem'] else x)
print(all_data_df['sentiment'].unique())
display(all_data_df)
all_data_df_2type=all_data_df[all_data_df['sentiment'].isin(['positive','negative'])].copy()
all_data_df=all_data_df.reset_index()
all_data_df_2type_neu=all_data_df[~all_data_df['sentiment'].isin(['positive','negative'])].copy()
all_data_df_2type['label']=all_data_df_2type['sentiment'].apply(lambda x: 1 if x=='positive' else 0)
all_data_df_2type['text_len']=all_data_df_2type['text'].apply(lambda x: len(x))
all_data_df_2type_pos=all_data_df_2type[all_data_df_2type['sentiment']=='positive'].copy()
all_data_df_2type_neg=all_data_df_2type[all_data_df_2type['sentiment']=='negative'].copy()
print(all_data_df['sentiment'].unique())
all_data_df_2type

['neutral' 'negative' 'positive']


,text,sentiment
1,Could you catch us up on your latest thinking ...,neutral
10,We continue to ramp up the activities for the ...,neutral
...,...,...
98,And so it's -- that one is a little bit toughe...,positive
99,"Thanks, Simeon. Rob, next question please?",neutral


['neutral' 'negative' 'positive']


,text,sentiment,label,text_len
100,"Yes, it was sort of just how the conversation ...",negative,0,310
110,"Well, it's still kind of early introduction, r...",positive,1,320
...,...,...,...,...
96,"Yes, Simeon. I still remain -- feel good about...",positive,1,256
98,And so it's -- that one is a little bit toughe...,positive,1,224


In [284]:
all_data_df_2type['text_len'].describe()

count      98.000
mean      534.459
           ...   
75%       652.250
max      1181.000
Name: text_len, Length: 8, dtype: float64

# 1. Problem 1

In [268]:
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(all_data_df_2type_pos[['text']], all_data_df_2type_pos[['label']], test_size=0.2, random_state=42)
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(all_data_df_2type_neg[['text']], all_data_df_2type_neg[['label']], test_size=0.2, random_state=42)
#display(X_train_p, X_test_p, y_train_p, y_test_p)
#display(X_train_n, X_test_n, y_train_n, y_test_n)
X_train=X_train_p.append(X_train_n).copy().reset_index(drop=True)
X_test=X_test_p.append(X_test_n).copy().reset_index(drop=True)
y_train=y_train_p.append(y_train_n).copy().reset_index(drop=True)
y_test=y_test_p.append(y_test_n).copy().reset_index(drop=True)

docs = all_data_df_2type['text'].tolist()
max_words = 10000 
t = Tokenizer(um_words=max_words)
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
for vector_len in [12]:
    for m_len in [100]:
        print("max length: "+str(m_len)+"   |   vector length: "+str(vector_len))
        x_train_doc=create_padded_docs(t,X_train['text'].tolist(),m_len)
        x_test_doc=create_padded_docs(t,X_test['text'].tolist(),m_len)
        y_train_label=y_train['label'].tolist()
        y_test_label=y_test['label'].tolist()

        model = Sequential()
        model.add(Embedding(vocab_size, vector_len, input_length=m_len))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        # compile the model
        model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
        # summarize the model
        print(model.summary())
        # fit the model
        model.fit(x_train_doc, y_train_label, epochs=50)#,validation_split=0.1)
        # evaluate the model
        loss, accuracy = model.evaluate(x_train_doc, y_train_label, verbose=0)
        print('Training Accuracy: %f' % (accuracy*100))
        y_train_label_predicted=model.predict(x_train_doc)
        y_test_label_predicted=model.predict(x_test_doc)
        y_train_label_predicted=[1 if y>=0.5 else 0 for y in y_train_label_predicted]
        y_test_label_predicted=[1 if y>=0.5 else 0 for y in y_test_label_predicted]
        loss, accuracy = model.evaluate(x_test_doc, y_test_label, verbose=0)
        print('Testing Accuracy: %f' % (accuracy*100))
        print('***'*30)

max length: 100   |   vector length: 12
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_133 (Embedding)    (None, 100, 12)           62532     
_________________________________________________________________
flatten_133 (Flatten)        (None, 1200)              0         
_________________________________________________________________
dense_133 (Dense)            (None, 1)                 1201      
Total params: 63,733
Trainable params: 63,733
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
647/647 [==============================] - 3s 5ms/step - loss: 0.6150 - acc: 0.7790
Epoch 2/50
647/647 [==============================] - 0s 94us/step - loss: 0.4998 - acc: 0.8083
Epoch 3/50
647/647 [==============================] - 0s 95us/step - loss: 0.4640 - acc: 0.8083
Epoch 4/50
647/647 [==============================] - 0s 93us/step - l

In [269]:
confusion_columns=['predicted_neg','predicted_pos']
confusion_rows=['true_neg','true_pos']
train_confusion_matrix=pd.DataFrame(confusion_matrix(y_test_label, y_test_label_predicted))
train_confusion_matrix.columns=confusion_columns
train_confusion_matrix['index']=confusion_rows
train_confusion_matrix=train_confusion_matrix.set_index('index')
train_confusion_matrix.index.name=''

test_confusion_matrix=pd.DataFrame(confusion_matrix(y_test_label, y_test_label_predicted))
test_confusion_matrix.columns=confusion_columns
test_confusion_matrix['index']=confusion_rows
test_confusion_matrix=test_confusion_matrix.set_index('index')
test_confusion_matrix.index.name=''

display(train_confusion_matrix)
display(test_confusion_matrix)

,predicted_neg,predicted_pos
,,
true_neg,1,30
true_pos,0,131


,predicted_neg,predicted_pos
,,
true_neg,1,30
true_pos,0,131


# 2. Problem 2

In [285]:
import os

imdb_dir = 'aclImdb'
def load_imdb_dir(imdb_dir,sub_folder):
    train_dir = os.path.join(imdb_dir,sub_folder)
    labels = []
    texts = []
    for label_type in ['neg', 'pos']:
        dir_name = os.path.join(train_dir, label_type)
        for fname in os.listdir(dir_name):
            if fname[-4:] == '.txt':
                f = open(os.path.join(dir_name, fname))
                texts.append(f.read())
                f.close()
                if label_type == 'neg':
                    labels.append(0)
                else:
                    labels.append(1)
    return texts,labels
x_train_imdb_raw,y_train_imdb_raw=load_imdb_dir(imdb_dir,"train")
x_test_imdb_raw,y_test_imdb_raw=load_imdb_dir(imdb_dir,"test")


x_train_imdb_raw_tot=x_train_imdb_raw+x_test_imdb_raw
y_train_imdb_raw_tot=y_train_imdb_raw+y_test_imdb_raw

max_words = 10000 
docs = all_data_df_2type['text'].tolist()+x_train_imdb_raw_tot
t = Tokenizer(num_words=max_words)
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

In [287]:

for vector_len in [12]:
    for m_len in [100]:
        print("max length: "+str(m_len)+"   |   vector length: "+str(vector_len))
        x_train_doc=create_padded_docs(t,x_train_imdb_raw_tot,m_len)
        x_test_doc=create_padded_docs(t,all_data_df_2type['text'].tolist(),m_len)
        y_train_label=y_train_imdb_raw_tot
        y_test_label=all_data_df_2type['label'].tolist()

        model = Sequential()
        model.add(Embedding(vocab_size, vector_len, input_length=m_len))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        # compile the model
        model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
        # summarize the model
        print(model.summary())
        # fit the model
        model.fit(x_train_doc, y_train_label, epochs=5,validation_split=0.1)
        # evaluate the model
        loss, accuracy = model.evaluate(x_train_doc, y_train_label, verbose=0)
        print('Training Accuracy: %f' % (accuracy*100))
        y_train_label_predicted=model.predict(x_train_doc)
        y_test_label_predicted=model.predict(x_test_doc)
        y_train_label_predicted=[1 if y>=0.5 else 0 for y in y_train_label_predicted]
        y_test_label_predicted=[1 if y>=0.5 else 0 for y in y_test_label_predicted]
        loss, accuracy = model.evaluate(x_test_doc, y_test_label, verbose=0)
        print('Testing Accuracy: %f' % (accuracy*100))
        print('***'*30)

max length: 100   |   vector length: 12
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_144 (Embedding)    (None, 100, 12)           1492164   
_________________________________________________________________
flatten_144 (Flatten)        (None, 1200)              0         
_________________________________________________________________
dense_144 (Dense)            (None, 1)                 1201      
Total params: 1,493,365
Trainable params: 1,493,365
Non-trainable params: 0
_________________________________________________________________
None
Train on 45000 samples, validate on 5000 samples
Epoch 1/5
45000/45000 [==============================] - 23s 519us/step - loss: 0.4630 - acc: 0.7714 - val_loss: 0.3638 - val_acc: 0.8390
Epoch 2/5
45000/45000 [==============================] - 20s 445us/step - loss: 0.2850 - acc: 0.8818 - val_loss: 0.3492 - val_acc: 0.8452
Epoch 3/5
45000/45000 [===

In [297]:
all_data_df_2type['y_predicted']=y_test_label_predicted
all_data_df_2type['label_predicted']=all_data_df_2type['y_predicted'].apply(lambda x: 'positive' if x==1 else "negative")
all_data_df_2type_neu['label_predicted']='neutral'
all_data_df_2type=all_data_df_2type.drop('y_predicted',axis=1)
ge_df=all_data_df_2type.append(all_data_df_2type_neu)
ge_df.to_json('GE_predicted.json',orient='split')


/Users/lovelife/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [308]:
output='''{"text":{'''
senti='''},"sentiment":{'''
for index, row in ge_df.iterrows():
    output+='''"'''+str(index)+'''":"'''+row['text']+'''",'''
    senti+='''"'''+str(index)+'''":"'''+row['label_predicted']+'''",'''
output=output[:-1]
senti=senti[:-1]+'}}'
out_senti=output+senti
out_senti

'{"text":{"1":"Good day, ladies and gentlemen and welcome to the General Electric Fourth Quarter 2018 Earnings Conference Call. At this time, all participants are in a listen-only mode. My name is Brandon and I\'ll be your conference facilitator today. [Operator Instructions] As a reminder, this conference is being recorded","10":"For the industrial portfolio in total, we will provide you with our outlook for organic growth and we\'ll do so directionally on operating margins and free cash flow. We will give you a more detailed outlook in the near term but not today. For Power, we are resetting the baseline. We are reviewing every single project and contract and digging deeper to understand costs and benefits with respect to restructuring, market and commercial execution, improvement opportunities, legacy project issues and our service operations. We\'re gaining more meaningful insights into the paths for near and long-term earnings and cash potential of this business. This work takes t

In [309]:
text_file = open("GE_predicted.json", "w")
text_file.write(out_senti)
text_file.close()

In [288]:
confusion_columns=['predicted_neg','predicted_pos']
confusion_rows=['true_neg','true_pos']
train_confusion_matrix=pd.DataFrame(confusion_matrix(y_test_label, y_test_label_predicted))
train_confusion_matrix.columns=confusion_columns
train_confusion_matrix['index']=confusion_rows
train_confusion_matrix=train_confusion_matrix.set_index('index')
train_confusion_matrix.index.name=''

test_confusion_matrix=pd.DataFrame(confusion_matrix(y_test_label, y_test_label_predicted))
test_confusion_matrix.columns=confusion_columns
test_confusion_matrix['index']=confusion_rows
test_confusion_matrix=test_confusion_matrix.set_index('index')
test_confusion_matrix.index.name=''

display(train_confusion_matrix)
display(test_confusion_matrix)

,predicted_neg,predicted_pos
,,
true_neg,11,13
true_pos,24,50


,predicted_neg,predicted_pos
,,
true_neg,11,13
true_pos,24,50


# Problem 3

IBM

In [ ]:
from watson_developer_cloud import NaturalLanguageUnderstandingV1

natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='{version}',
    username='{username}',
    password='{password}',
    url='{url}'
)

In [396]:
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
    import Features, EntitiesOptions, KeywordsOptions

naturalLanguageUnderstanding = NaturalLanguageUnderstandingV1(
    version='2018-11-16',
    username='wenjun song',
    password='Ilove22*',
    #iam_apikey='OAbro0YFtCjxCW5Z8EiHjBcKGHtm2qpl30rvUw9yDZJ1',
    url='https://gateway-tok.watsonplatform.net/natural-language-understanding/api')

response = naturalLanguageUnderstanding.analyze(
    text=all_data_df_2type['text'].tolist()[0],
    features=Features(
        entities=EntitiesOptions(emotion=True, sentiment=True, limit=2),
        keywords=KeywordsOptions(emotion=True, sentiment=True,
                                 limit=2))).get_result()

print(json.dumps(response, indent=2))

WatsonApiException: Error: Unauthorized: Access is denied due to invalid credentials , Code: 401

# Google

In [313]:
!pip install --upgrade google-api-python-client
!easy_install --upgrade google-api-python-client
! pip install google-cloud-storage
! pip install google-cloud-language
! pip install --upgrade google-cloud-language
import argparse
import google.cloud
from google.cloud import language
from google.cloud import storage
from google.cloud.language import enums
from google.cloud.language import types
import os
from google.oauth2 import service_account
!pip install google-auth-oauthlib
from google_auth_oauthlib.flow import Flow
from google_auth_oauthlib.flow import InstalledAppFlow

    100% |████████████████████████████████| 61kB 1.8MB/s ta 0:00:01
    100% |████████████████████████████████| 225kB 6.6MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 6.4MB/s ta 0:00:011
    100% |████████████████████████████████| 71kB 7.3MB/s ta 0:00:011
    100% |████████████████████████████████| 81kB 8.0MB/s ta 0:00:011
  Running setup.py bdist_wheel for httplib2 ... done
  Stored in directory: /Users/lovelife/Library/Caches/pip/wheels/98/82/0d/cfb126a5e40d487157e43fdb3332937713dde36b4d0e1e754d
Successfully built httplib2
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Searching for google-api-python-client
Reading https://pypi.org/simple/google-api-python-client/
Best match: google-api-python-client 1.7.8
Processing google_api_python_client-1.7.8-py3-none-any.whl
Installing google_api_python_client-1.7.8-py3-none-any.whl to /Users/lovelife/anaconda3/lib/python3.6/site-

  Running setup.py bdist_wheel for googleapis-common-protos ... done
  Stored in directory: /Users/lovelife/Library/Caches/pip/wheels/51/ef/76/6ca53de13cc85eea40c641d75bb01355d97181ed64eb67bc8e
Successfully built googleapis-common-protos
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 71kB 1.8MB/s ta 0:00:01
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: google-cloud-language in /Users/lovelife/anaconda3/lib/python3.6/site-packages (1.1.1)


You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/c2/e2/9fd03d55ffb70fe51f587f20bcf407a6927eb121de86928b34d162f0b1ac/requests_oauthlib-1.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/16/95/699466b05b72b94a41f662dc9edf87fda4289e3602ecd42d27fcaddf7b56/oauthlib-3.0.1-py2.py3-none-any.whl
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [342]:

cred_file_loc = r'dogwood-dryad-235122-9721a66a61c1.json'
cred = service_account.Credentials.from_service_account_file(cred_file_loc)
Client = language.LanguageServiceClient(credentials=cred)
paragraphs = all_data_df['text'].tolist()
results = []
for paragraph in paragraphs:
    document = types.Document(content=paragraph, type=enums.Document.Type.PLAIN_TEXT)
    result = Client.analyze_sentiment(document=document)
    results.append(result)

results[11]

document_sentiment {
  magnitude: 0.30000001192092896
}
language: "en"
sentences {
  text {
    content: "Can I ask just sort of quick one?"
    begin_offset: -1
  }
  sentiment {
    magnitude: 0.20000000298023224
    score: 0.20000000298023224
  }
}
sentences {
  text {
    content: "Just for Greg."
    begin_offset: -1
  }
  sentiment {
  }
}
sentences {
  text {
    content: "Greg, when we think about the 727 engine versus the MAX, just for modeling purposes it\'s going to review 90% MAXs this year."
    begin_offset: -1
  }
  sentiment {
  }
}
sentences {
  text {
    content: "How do we think about the margin profile of one variant versus the other?"
    begin_offset: -1
  }
  sentiment {
  }
}

In [391]:
def classify_google(x):
    if x['score']<=-0.10:
        return 'negative'
    elif x['score']>-0.10 and (x['score']<0.25):
        return 'neutral'
    else:
        return 'positive'

magnitudes=[]
scores=[]
for i in range(len(results)):
    magnitudes.append(results[i].document_sentiment.magnitude)
    scores.append(results[i].document_sentiment.score)
google_sentiment=pd.DataFrame({'magnitude':magnitudes,'score':scores})
google_sentiment['label_goog']=google_sentiment.apply(classify_google,axis=1)
google_merged=pd.concat([all_data_df,google_sentiment],axis=1)
test_confusion_matrix=pd.DataFrame(confusion_matrix(google_merged['sentiment'].tolist(), google_merged['label_goog'].tolist()))
test_confusion_matrix.columns=confusion_columns=['predicted_neg','predicted_neutral','predicted_pos']
test_confusion_matrix['index']=['true_neg','true_neutral','true_pos']
test_confusion_matrix=test_confusion_matrix.set_index('index')
test_confusion_matrix.index.name=''

display(test_confusion_matrix)

,predicted_neg,predicted_neutral,predicted_pos
,,,
true_neg,23,105,27
true_neutral,56,639,140
true_pos,25,329,300


# Amazon

In [400]:
for index, row in all_data_df.iterrows():
    text_file = open("Amazon/text_"+str(index)+".txt", "w")
    text_file.write(row['text'])
    text_file.close()

In [426]:
amazon_predict=pd.read_csv('amazon_prediction.csv')
amazon_predict['MyId']=amazon_predict['imagelocation'].apply(lambda x: int(x.split('_')[1].split('.')[0]))
amazon_predict=amazon_predict.sort_values(['MyId'])
amazon_predict=amazon_predict[['sentiment','positive','negative','neutral','mixed']]
amazon_predict['label_amazon_predicted']=amazon_predict['sentiment'].apply(lambda x: x.lower())
amazon_predict['label_amazon_predicted']=amazon_predict['label_amazon_predicted'].apply(lambda x: 'positive' if x=='mixed' else x)
amazon_predict=amazon_predict.drop('sentiment',axis=1)
amazon_predict

,positive,negative,neutral,mixed,label_amazon_predicted
985,0.008,2.155e-01,0.754,0.022,neutral
437,0.135,5.293e-04,0.862,0.002,neutral
...,...,...,...,...,...
1638,0.340,3.178e-02,0.083,0.545,positive
1425,0.058,3.285e-02,0.902,0.007,neutral


In [428]:
amzn_merged=pd.concat([all_data_df,amazon_predict],axis=1)
test_confusion_matrix=pd.DataFrame(confusion_matrix(amzn_merged['sentiment'].tolist(), amzn_merged['label_amazon_predicted'].tolist()))
test_confusion_matrix.columns=confusion_columns=['predicted_neg','predicted_neutral','predicted_pos']
test_confusion_matrix['index']=['true_neg','true_neutral','true_pos']
test_confusion_matrix=test_confusion_matrix.set_index('index')
test_confusion_matrix.index.name=''

display(test_confusion_matrix)

,predicted_neg,predicted_neutral,predicted_pos
,,,
true_neg,12,115,28
true_neutral,42,553,240
true_pos,41,524,89


In [393]:
! pip install auto-sklearn

  Using cached https://files.pythonhosted.org/packages/c3/c6/c555cfa3c7d0078dded091d4901ed52344bbb925077aa70b871faf35fd58/pyrfr-0.7.4.tar.gz
  Using cached https://files.pythonhosted.org/packages/60/b4/4df37087a1d36755e3a3bfd2a30263f358d2dea21938240fa02313d45f51/sphinx_rtd_theme-0.4.3-py2.py3-none-any.whl
  Complete output from command /Users/lovelife/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/h1/pcv8265963799lkgn7tjtctw0000gn/T/pip-install-01_0z54l/pyrfr/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /private/var/folders/h1/pcv8265963799lkgn7tjtctw0000gn/T/pip-wheel-q344oxot --python-tag cp36:
  running bdist_wheel
  running build
  running build_ext
  building 'pyrfr._regression' extension
  swigging pyrfr/regression.i to pyrfr/regression_wrap.cpp
  swig -python -c++ -modern -features nondynamic -I./include -o pyrfr/regression_wrap.cpp

Command "/Users/lovelife/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/h1/pcv8265963799lkgn7tjtctw0000gn/T/pip-install-01_0z54l/pyrfr/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /private/var/folders/h1/pcv8265963799lkgn7tjtctw0000gn/T/pip-record-kv1hqkrw/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /private/var/folders/h1/pcv8265963799lkgn7tjtctw0000gn/T/pip-install-01_0z54l/pyrfr/


In [317]:
! pip install -U scikit-learn

    100% |████████████████████████████████| 8.0MB 4.0MB/s ta 0:00:011
autokeras 0.3.7 has requirement numpy==1.15.4, but you'll have numpy 1.14.5 which is incompatible.
autokeras 0.3.7 has requirement scikit-learn==0.20.2, but you'll have scikit-learn 0.20.3 which is incompatible.
  Found existing installation: scikit-learn 0.19.2
    Uninstalling scikit-learn-0.19.2:
      Successfully uninstalled scikit-learn-0.19.2
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [319]:
! pip install numpy

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [320]:
! pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 5.9MB/s ta 0:00:011
  Found existing installation: pip 18.1
    Uninstalling pip-18.1:
      Successfully uninstalled pip-18.1


In [321]:
! pip install -U numpy

    100% |████████████████████████████████| 13.9MB 2.7MB/s ta 0:00:011
autokeras 0.3.7 has requirement numpy==1.15.4, but you'll have numpy 1.16.2 which is incompatible.
autokeras 0.3.7 has requirement scikit-learn==0.20.2, but you'll have scikit-learn 0.20.3 which is incompatible.
  Found existing installation: numpy 1.14.5
    Uninstalling numpy-1.14.5:
      Successfully uninstalled numpy-1.14.5
